In [1]:
import os
import pandas as pd
from PIL import Image

# Specify the path to your image dataset folder
dataset_folder = '/content/drive/MyDrive/line road images'

# Create empty lists to store image paths and labels
image_paths = []
labels = []

# Loop through the dataset folder and retrieve image paths and labels
for class_label in os.listdir(dataset_folder):
    class_folder = os.path.join(dataset_folder, class_label)
    if os.path.isdir(class_folder):
        for image_file in os.listdir(class_folder):
            image_path = os.path.join(class_folder, image_file)
            image_paths.append(image_path)
            labels.append(class_label)

# Create a DataFrame with image paths and labels
data = {'Image_Path': image_paths, 'Label': labels}
df = pd.DataFrame(data)

# Save the DataFrame as a CSV file
df.to_csv('image_dataset.csv', index=False)

In [ ]:
import os
import pandas as pd
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam

# Specify the desired width and height for the images
image_width = 200
image_height = 200

# Load the dataset from a CSV file
dataset = pd.read_csv('/content/image_dataset.csv')

# Load and preprocess the image data
X = []
y = []
for index, row in dataset.iterrows():
    image_path = row['Image_Path']
    if os.path.isdir(image_path):
        print(f"Skipping directory: {image_path}")
        continue
    try:
        image = Image.open(image_path)
        image = image.resize((image_width, image_height))  # Resize the image to a consistent size
        image = np.array(image) / 255.0  # Normalize pixel values to [0, 1]
    except Exception as e:
        print(f"Error loading image: {image_path}. Skipping...")
        print(f"Error message: {str(e)}")
        continue
    if image.shape != (image_width, image_height, 3):
        print(f"Inconsistent shape for image: {image_path}. Skipping...")
        continue
    X.append(image)
    y.append(row['Label'])

X = np.array(X)
y = np.array(y)

# Encode class labels to integers
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert class labels to one-hot encoded vectors
num_classes = len(np.unique(y))
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# Create the CNN model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(image_width, image_height, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))  # Apply dropout regularization
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])



Skipping directory: /content/drive/MyDrive/line road images/Broken yellow line road images/.ipynb_checkpoints
Skipping directory: /content/drive/MyDrive/line road images/Single yellow line road images/.ipynb_checkpoints


In [ ]:
# Print the model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 198, 198, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 99, 99, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 97, 97, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 48, 48, 64)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 147456)            0         
                                                                 
 dense (Dense)               (None, 128)               1

In [ ]:

# Train the model
model.fit(X_train, y_train,
          batch_size=32,
          epochs=10,
          validation_data=(X_test, y_test))

# Evaluate the model
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/10
1/1 [==============================] - 4s 4s/step - loss: 1.8196 - accuracy: 0.0938 - val_loss: 12.5626 - val_accuracy: 0.1250
Epoch 2/10
1/1 [==============================] - 4s 4s/step - loss: 11.0291 - accuracy: 0.1875 - val_loss: 11.6775 - val_accuracy: 0.1250
Epoch 3/10
1/1 [==============================] - 2s 2s/step - loss: 8.5690 - accuracy: 0.2812 - val_loss: 10.6777 - val_accuracy: 0.1250
Epoch 4/10
1/1 [==============================] - 2s 2s/step - loss: 8.2277 - accuracy: 0.2188 - val_loss: 4.6271 - val_accuracy: 0.1250
Epoch 5/10
1/1 [==============================] - 2s 2s/step - loss: 4.9202 - accuracy: 0.1875 - val_loss: 2.5777 - val_accuracy: 0.1250
Epoch 6/10
1/1 [==============================] - 2s 2s/step - loss: 2.7987 - accuracy: 0.2500 - val_loss: 2.4996 - val_accuracy: 0.1250
Epoch 7/10
1/1 [==============================] - 4s 4s/step - loss: 2.0539 - accuracy: 0.3438 - val_loss: 1.5530 - val_accuracy: 0.2500
Epoch 8/10
1/1 [=====================

In [ ]:
from PIL import Image
import numpy as np

# Load and preprocess a new image
image_path = '/content/img1.jpeg'  # Replace with the path to your new image
image_width = 200
image_height = 200

image = Image.open(image_path)
image = image.resize((image_width, image_height))
image = np.array(image) / 255.0
image = np.expand_dims(image, axis=0)  # Add an extra dimension to match the input shape of the model

# Make predictions using your trained model
predictions = model.predict(image)
predicted_class = np.argmax(predictions)

# Convert the predicted class index back to the original label
predicted_label = label_encoder.inverse_transform([predicted_class])[0]

print('Predicted class:', predicted_class)
print('Predicted label:', predicted_label)

#meaning road sign
print("The meaning of", predicted_label ,"Lane:")
if predicted_class == 0:
    print("You can cross the line to change lanes if it is safe to do so.")
elif predicted_class == 1:
    print("You can pass a slower moving vehicle on the left side if it is safe to do so.")
elif predicted_class == 2:
    print("You cannot cross the line to change lanes or pass a slower moving vehicle. This is because double yellow lines indicate that there is a no-passing zone.")
elif predicted_class == 3:
    print("You cannot cross the line to change lanes or pass a slower moving vehicle. This is because double white lines indicate that there is a no-passing zone.")
elif predicted_class == 4:
    print("Indicates that you cannot cross the line to change lanes.")
elif predicted_class == 5:
    print("Indicates that you cannot pass a slower moving vehicle on the left side.")
else:
    print("Sorry, this sign is not added.")



1/1 [==============================] - 0s 57ms/step
Predicted class: 0
Predicted label: Broken White line road images
The meaning of Broken White line road images Lane:
You can cross the line to change lanes if it is safe to do so.
